In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [47]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk

In [48]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [49]:
# Import Dataset
df = pd.read_json('../data/newspaper/prothom_alo_2019-09-03.txt')
print(df.content.unique())
df.head()

['\nরোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ: যথাযথ পদক্ষেপ নেবে অ্যামটব\n -> \nমুঠোফোন অপারেটরদের সংগঠন অ্যাসোসিয়েশন অব মোবাইল টেলিকম অপারেটরস অব বাংলাদেশ (অ্যামটব) বলেছে, জাতীয় পরিচয় তথ্যভান্ডারে (এনআইডি) সংরক্ষিত তথ্যের সঙ্গে আঙুলের ছাপ যাচাই করার পরই কেবল মোবাইলের সিম সক্রিয় হয়।\nআজ রোববার বাংলাদেশ টেলিযোগাযোগ নিয়ন্ত্রণ কমিশন (বিটিআরসি) মোবাইল অপারেটরদের চিঠি দিয়ে রোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ দেওয়ার পর এক প্রতিক্রিয়ায় এ কথা জানায় অ্যামটব।\nঅ্যামটবের মহাসচিব এস এম ফরহাদ বলেন, ‘মোবাইল অপারেটরেরা সব সময়ই বিটিআরসির নির্দেশনা মেনে চলে। এ ব্যাপারে তারা নিজেদের আয়ত্তের মধ্যে সম্ভব যথাযথ পদক্ষেপ নেবে।’ তিনি আরও বলেন, ‘আরেকটি বিষয় বলা দরকার, এনআইডি তথ্যভান্ডারের সঙ্গে আঙুলের ছাপ মিলিয়ে নিশ্চিত হওয়ার পরেই কেবল মোবাইল সিম সক্রিয় করা হয়।’\nবিটিআরসির চিঠিতে বলা হয়েছে, রোহিঙ্গা ক্যাম্পে কোনো প্রকার সিম বিক্রি করা যাবে না। পাশাপাশি রোহিঙ্গাদের মুঠোফোন সুবিধা দেওয়ার বিষয়েও নিষেধ করেছে বিটিআরসি। এ বিষয়ে আগামী সাত কার্যদিবসের মধ্যে ব্যবস্থা নিয়ে বিটিআরসিকে জানাতে বলা হয়েছে।\nএক বিজ্ঞপ্তিতে

,content
0,\nরোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ: যথা...
1,\nরাবিতে ভর্তি পরীক্ষার আবেদন শুরু কাল থেকে\n ...
10,\nমাকে পুড়িয়ে মারার অভিযোগে মাদকাসক্ত ছেলের বি...
11,\nসন্ধ্যার মধ্যেই জামিনে মুক্তি পাচ্ছেন আয়শা\...
12,\nআরামে সহজে ফ্যাশনে\n -> \n\n\nফ্যাশন আর আরাম...


In [50]:
# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

[' রোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ: যথাযথ পদক্ষেপ নেবে অ্যামটব -> '
 'মুঠোফোন অপারেটরদের সংগঠন অ্যাসোসিয়েশন অব মোবাইল টেলিকম অপারেটরস অব বাংলাদেশ '
 '(অ্যামটব) বলেছে, জাতীয় পরিচয় তথ্যভান্ডারে (এনআইডি) সংরক্ষিত তথ্যের সঙ্গে '
 'আঙুলের ছাপ যাচাই করার পরই কেবল মোবাইলের সিম সক্রিয় হয়। আজ রোববার বাংলাদেশ '
 'টেলিযোগাযোগ নিয়ন্ত্রণ কমিশন (বিটিআরসি) মোবাইল অপারেটরদের চিঠি দিয়ে '
 'রোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ দেওয়ার পর এক প্রতিক্রিয়ায় এ কথা '
 'জানায় অ্যামটব। অ্যামটবের মহাসচিব এস এম ফরহাদ বলেন, ‘মোবাইল অপারেটরেরা সব '
 'সময়ই বিটিআরসির নির্দেশনা মেনে চলে। এ ব্যাপারে তারা নিজেদের আয়ত্তের মধ্যে '
 'সম্ভব যথাযথ পদক্ষেপ নেবে।’ তিনি আরও বলেন, ‘আরেকটি বিষয় বলা দরকার, এনআইডি '
 'তথ্যভান্ডারের সঙ্গে আঙুলের ছাপ মিলিয়ে নিশ্চিত হওয়ার পরেই কেবল মোবাইল সিম '
 'সক্রিয় করা হয়।’ বিটিআরসির চিঠিতে বলা হয়েছে, রোহিঙ্গা ক্যাম্পে কোনো প্রকার '
 'সিম বিক্রি করা যাবে না। পাশাপাশি রোহিঙ্গাদের মুঠোফোন সুবিধা দেওয়ার বিষয়েও '
 'নিষেধ করেছে বিটিআরসি। এ বিষয়ে আগামী সাত কার্যদিবসের মধ্যে ব্যবস্থা নিয়ে '
 '

In [51]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(sentence))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:2])

[['রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', ':', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'অ্যামটব', '-', '>', 'মুঠোফোন', 'অপারেটরদের', 'সংগঠন', 'অ্যাসোসিয়েশন', 'অব', 'মোবাইল', 'টেলিকম', 'অপারেটরস', 'অব', 'বাংলাদেশ', '(', 'অ্যামটব', ')', 'বলেছে', ',', 'জাতীয়', 'পরিচয়', 'তথ্যভান্ডারে', '(', 'এনআইডি', ')', 'সংরক্ষিত', 'তথ্যের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'যাচাই', 'করার', 'পরই', 'কেবল', 'মোবাইলের', 'সিম', 'সক্রিয়', 'হয়।', 'আজ', 'রোববার', 'বাংলাদেশ', 'টেলিযোগাযোগ', 'নিয়ন্ত্রণ', 'কমিশন', '(', 'বিটিআরসি', ')', 'মোবাইল', 'অপারেটরদের', 'চিঠি', 'দিয়ে', 'রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'দেওয়ার', 'পর', 'এক', 'প্রতিক্রিয়ায়', 'এ', 'কথা', 'জানায়', 'অ্যামটব।', 'অ্যামটবের', 'মহাসচিব', 'এস', 'এম', 'ফরহাদ', 'বলেন', ',', '‘', 'মোবাইল', 'অপারেটরেরা', 'সব', 'সময়ই', 'বিটিআরসির', 'নির্দেশনা', 'মেনে', 'চলে।', 'এ', 'ব্যাপারে', 'তারা', 'নিজেদের', 'আয়ত্তের', 'মধ্যে', 'সম্ভব', 'যথাযথ', 'পদক্ষেপ', 'নেবে।', '’', 'তিনি', 'আরও', 'বলেন', ',', '‘', 'আরেকটি', 'বিষয়', 'বলা', 'দরকার', ',', 'এনআইডি', '

In [52]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', ':', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'অ্যামটব', '-_>', 'মুঠোফোন', 'অপারেটরদের', 'সংগঠন', 'অ্যাসোসিয়েশন', 'অব', 'মোবাইল', 'টেলিকম', 'অপারেটরস', 'অব', 'বাংলাদেশ', '(', 'অ্যামটব', ')', 'বলেছে', ',', 'জাতীয়', 'পরিচয়', 'তথ্যভান্ডারে', '(', 'এনআইডি', ')', 'সংরক্ষিত', 'তথ্যের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'যাচাই', 'করার', 'পরই', 'কেবল', 'মোবাইলের', 'সিম', 'সক্রিয়', 'হয়।', 'আজ', 'রোববার', 'বাংলাদেশ', 'টেলিযোগাযোগ', 'নিয়ন্ত্রণ', 'কমিশন', '(', 'বিটিআরসি', ')', 'মোবাইল', 'অপারেটরদের', 'চিঠি', 'দিয়ে', 'রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'দেওয়ার', 'পর', 'এক', 'প্রতিক্রিয়ায়', 'এ', 'কথা', 'জানায়', 'অ্যামটব।', 'অ্যামটবের', 'মহাসচিব', 'এস', 'এম', 'ফরহাদ', 'বলেন', ',', '‘', 'মোবাইল', 'অপারেটরেরা', 'সব', 'সময়ই', 'বিটিআরসির', 'নির্দেশনা', 'মেনে', 'চলে।', 'এ', 'ব্যাপারে', 'তারা', 'নিজেদের', 'আয়ত্তের', 'মধ্যে', 'সম্ভব', 'যথাযথ', 'পদক্ষেপ', 'নেবে।', '’', 'তিনি', 'আরও', 'বলেন', ',', '‘', 'আরেকটি', 'বিষয়', 'বলা', 'দরকার', ',', 'এনআইডি', 'তথ্য

In [53]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[(0, 3), (1, 3), (2, 11), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 1), (23, 3), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 2), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 2), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 3), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 3), (99, 5), (100, 1), (101, 2), (102, 2), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1

In [54]:
id2word[1]

')'

In [55]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('(', 3),
  (')', 3),
  (',', 11),
  ('-', 1),
  (':', 2),
  ('>', 1),
  ('অপারেটরদের', 2),
  ('অপারেটরস', 1),
  ('অপারেটরেরা', 1),
  ('অব', 2),
  ('অ্যামটব', 2),
  ('অ্যামটব।', 1),
  ('অ্যামটবের', 1),
  ('অ্যাসোসিয়েশন', 1),
  ('আইন', 1),
  ('আইনশৃঙ্খলা', 1),
  ('আগামী', 1),
  ('আগেও', 1),
  ('আঙুলের', 2),
  ('আজ', 1),
  ('আরও', 2),
  ('আরেকটি', 1),
  ('আয়ত্তের', 1),
  ('এ', 3),
  ('এই', 1),
  ('এক', 2),
  ('এনআইডি', 2),
  ('এম', 1),
  ('এস', 1),
  ('ও', 4),
  ('কথা', 1),
  ('কমিটি', 1),
  ('কমিশন', 2),
  ('করতে', 1),
  ('করা', 2),
  ('করার', 1),
  ('করেছে', 1),
  ('কার্যদিবসের', 1),
  ('কিন্তু', 1),
  ('কেবল', 2),
  ('কোনো', 1),
  ('ক্যাম্প', 1),
  ('ক্যাম্পে', 2),
  ('খবর', 1),
  ('গণমাধ্যমে', 1),
  ('গুরুত্ব', 1),
  ('গোয়েন্দা', 1),
  ('চলে।', 1),
  ('চিঠি', 2),
  ('চিঠিতে', 2),
  ('ছাপ', 2),
  ('জন', 1),
  ('জব্বারের', 1),
  ('জাতীয়', 1),
  ('জানাতে', 1),
  ('জানানো', 1),
  ('জানায়', 1),
  ('টেলিকম', 1),
  ('টেলিযোগাযোগ', 2),
  ('ডাক', 1),
  ('তথ্য', 1),
  ('তথ্যভান্ডারে', 1),
  

In [56]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [57]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.001*"," + 0.001*"ও" + 0.001*"করে" + 0.001*"’" + 0.001*"এই" + 0.001*"থেকে" '
  '+ 0.001*"সঙ্গে" + 0.001*"‘" + 0.001*"সময়" + 0.001*"আর"'),
 (1,
  '0.001*"," + 0.000*"ও" + 0.000*"সঙ্গে" + 0.000*"থেকে" + 0.000*"মধ্যে" + '
  '0.000*"করে" + 0.000*"করা" + 0.000*"বলা" + 0.000*"এই" + 0.000*"পর"'),
 (2,
  '0.016*"," + 0.011*"বাস" + 0.011*"মোরশেদ" + 0.010*"থেকে" + '
  '0.010*"ক্যাটাগরির" + 0.008*"লাইসেন্স" + 0.008*"তিনি" + 0.007*"চালানোর" + '
  '0.007*"পর" + 0.007*"প্রথম"'),
 (3,
  '0.000*"ও" + 0.000*"মামলা" + 0.000*"রিফাত" + 0.000*"করে" + 0.000*"," + '
  '0.000*"তাঁর" + 0.000*"থেকে" + 0.000*"আজ" + 0.000*"বরগুনা" + 0.000*"আদালতে"'),
 (4,
  '0.026*"," + 0.023*"সৌদি" + 0.012*"তিনি" + 0.012*"বলেন" + 0.012*"করেন।" + '
  '0.012*"বাংলাদেশের" + 0.012*"নৌপ্রধান" + 0.012*"প্রধানমন্ত্রী" + 0.009*"ও" + '
  '0.009*"বাংলাদেশ"'),
 (5,
  '0.011*"রিফাত" + 0.010*"করে" + 0.010*"তাঁর" + 0.010*"বরগুনা" + 0.008*"," + '
  '0.008*"এই" + 0.008*"এরপর" + 0.008*"আয়শা" + 0.008*"আদালতে" + 0.008*"আয়শার"'),
 (6,
  '0

In [58]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.5833823215569245

Coherence Score:  0.4160026081445835
